# Pickling of NGSolve objects

Python objects an be converted to byte-streams, which can be stored to files, and later the 
Python object can be reconstructed by the inverse process. In general this is known as serialization,
in Python it is called pickling, see
https://docs.python.org/3/library/pickle.html.
Another usecase is parallel computing, where it is used to send whole Python objects across a network.

Many of the NGSolve-objects support pickling.

In [ ]:
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

In [ ]:
import pickle
outfile = open("mesh.pkl", "wb")
pickle.dump(mesh, outfile)

In [ ]:
infile = open("mesh.pkl", "rb")
mesh2 = pickle.load(infile)
mesh2.nv, mesh2.ne

In [ ]:
from ngsolve.webgui import Draw
Draw (mesh2);

## Shared objects remain shared

When we create several spaces on the same mesh, all spaces link to the mesh via a shared pointer. Similarly, if we have several `GridFunction`s defined on the same space, they link to it using a shared pointer. These shared objects remain shared after pickling and unpickling:

In [ ]:
fes = H1(mesh, order=2)
gfu1 = GridFunction(fes)
gfu2 = GridFunction(fes)
gfu1.Set(x)
gfu2.Set(y)

outfile = open("gridfunction.pkl", "wb")
pickle.dump([gfu1,gfu2], outfile)

In [ ]:
infile = open("gridfunction.pkl", "rb")
gfv1,gfv2 = pickle.load(infile)
print ("the same spaces:", id(gfv1.space), "=?=", id(gfv2.space))

Draw (gfv1);

## Pickling expression trees

`CoefficientFunction` expression trees support pickling as well.

In [ ]:
func = x*gfu1 + y
print (func)

In [ ]:
outfile = open("func.pkl", "wb")
pickle.dump([mesh,func], outfile)

infile = open("func.pkl", "rb")
mesh2,func2 = pickle.load(infile)

print (func2)

Draw (func2, mesh2);

We were pickling the mesh explicitly to have it available for drawing, it would be contained in the function `func` anyway.  

## Implementation of pickling

[pybind11-pickling](https://pybind11.readthedocs.io/en/stable/advanced/classes.html?highlight=pickling#pickling-support) supports wrapping of serialization of user-classes. 

And then there is ngcore - archive ...